### Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
#### Задание 1.
Обязательная часть  
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.  

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:  

KEYWORDS = ['python', 'парсинг']  

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).  

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка> 

#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.  

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>  

In [27]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd

class Habr_articles:
    def __init__(self,
                 keywords=['python', 'парсинг'],
                 url='https://habr.com/ru/all/'
                ):
        self.url = url
        self.keywords = keywords
        self.soup = BeautifulSoup(req.get(self.url).text)
    
    def get_urls(self):
        try:
            return [link['href'] for link in self.soup.find_all('a', class_='post__title_link', href=True)]
        except:
            return []
    
    def get_articles_by_keywords(self):
        result_pd = pd.DataFrame()
        for url in self.get_urls():
            try:
                soup = Habr_articles(url=url).soup
                title = soup.find('span', class_='post__title-text').text
                date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
                text = soup.find('div', class_='post__body post__body_full').text
                for word in self.keywords:
                    if word.lower() in title.lower() or word.lower() in text.lower():
                        row = {'дата': date, 'заголовок': title, 'ссылка': url, 'текст_статьи' : text}
                        result_pd = pd.concat([result_pd, pd.DataFrame([row])])
            except:
                result_pd = pd.concat([result_pd, pd.DataFrame({'дата': 'Error', 'заголовок': 'Error', 'ссылка': 'Error', 'текст_статьи' : 'Error'})])
        return result_pd.drop_duplicates(keep=False)

In [2]:
Habr_articles().get_articles_by_keywords()

,дата,заголовок,ссылка,текст_статьи
0,2021-03-23,«Отдых — это непросто»: Андрей Бреслав о жизни...,https://habr.com/ru/company/jugru/blog/548408/,\nДля многих в IT-сообществе имя Андрея Бресла...
0,2021-03-23,Process Mining на базе BI — реальные возможнос...,https://habr.com/ru/company/visiology/blog/545...,"\nПривет, Хабр! Как вы считаете, в каких ситуа..."


### Задание 2.
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:  
EMAIL = [xxx@x.ru, yyy@y.com]  

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>  

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [28]:
import json
import time

In [126]:
def check_mail_hacked(emails):
    result_pd = pd.DataFrame()
    for email in emails:
        try:
            request = req.post(
                'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data', 
                data=json.dumps({"emailAddresses":[email]}),
                headers={
                    'Accept': 'application/json, text/plain, */*',
                    'Accept-Encoding': 'gzip, deflate, br',
                    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
                    'Connection': 'keep-alive',
                    'Content-Length': '37',
                    'Content-Type': 'application/json;charset=UTF-8',
                    'DNT': '1',
                    'Host': 'identityprotection.avast.com',
                    'Origin': 'https://www.avast.com',
                    'Referer': 'https://www.avast.com/',
                    'sec-ch-ua': 'Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99',
                    'sec-ch-ua-mobile': '?0',
                    'Sec-Fetch-Dest': 'empty',
                    'Sec-Fetch-Mode': 'cors',
                    'Sec-Fetch-Site': 'same-site',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
                    'Vaar-Header-App-Build-Version': '1.0.0',
                    'Vaar-Header-App-Product': 'hackcheck-web-avast',
                    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
                    'Vaar-Version': '0'
            }
            ).json()
            
            result  = pd.DataFrame(request['breaches']).T.reset_index()
            result = result[['publishDate', 'site', 'description']]
            result.columns = [ 'дата утечки', 'источник утечки', 'описание утечки']
            result['дата утечки'] = pd.to_datetime(result['дата утечки']).dt.date
            result.loc[result['дата утечки'] != 0, 'почта'] = email
            result_pd = pd.concat([result_pd, result])
            
            time.sleep(1)
        except:
            result_pd = pd.concat([result_pd, pd.DataFrame({'дата утечки': 'нет данных', 
                                                            'источник утечки': 'нет данных', 
                                                            'описание утечки': 'нет данных', 
                                                            'почта': email}, index=[0])
                                  ])
            time.sleep(1)
    return result_pd

In [129]:
check_mail_hacked(['sidorov@ya.ru', 'petrov@ya.ru', 'ivanov@ya.ru'])

,дата утечки,источник утечки,описание утечки,почта
0,2016-10-29,vk.com,Popular Russian social networking platform VKo...,sidorov@ya.ru
0,2020-12-03,cryptopayeer.fun,"In November 2020, a collection of over 23,000 ...",petrov@ya.ru
1,2016-10-24,000webhost.com,An anonymous criminal was able to exploit an o...,petrov@ya.ru
2,2016-11-01,qip.ru,"In 2011, Russian instant messaging service pro...",petrov@ya.ru
3,2020-12-10,schoeftland.swiss,"In November 2020, a collection of over 23,000 ...",petrov@ya.ru
4,2016-10-29,vk.com,Popular Russian social networking platform VKo...,petrov@ya.ru
5,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe...",petrov@ya.ru
0,2018-11-21,animoto.com,"In July 2018, the online video site Animoto su...",ivanov@ya.ru
1,2016-11-01,qip.ru,"In 2011, Russian instant messaging service pro...",ivanov@ya.ru
2,2020-12-03,indexbox.ru,"In November 2020, a collection of over 23,000 ...",ivanov@ya.ru
